In [1]:
import pandas as pd
from scipy.interpolate import CubicSpline
import numpy as np

In [2]:
### Read Data into single df
all_sofr = pd.read_excel('C:/Users/Aparna/Documents/Fixed Income Project/sofr_data_full.xlsx',sheet_name = None)
df_list = []
for frame in all_sofr.keys():
    temp = all_sofr[frame]
    temp = temp.dropna()
    temp = temp.reindex(index=temp.index[::-1]).reset_index().iloc[:,2:4]
    temp.set_index('dates', inplace = True)
    df_list.append(temp)
sofr_df = pd.concat(df_list, axis = 1)
sofr_df = sofr_df.dropna()
sofr_df.columns = list(all_sofr.keys())
sofr_df.rename(columns = {'1y':'12m'}, inplace = True)
sofr_df.head(2)

,1m,3m,6m,9m,12m,2y,3y,5y,7y,10y,15y,20y,30y
dates,,,,,,,,,,,,,
2008-09-19,1.9416,1.9752,1.9847,2.0446,2.1091,2.5107,2.9136,3.4175,3.7125,3.9847,4.2585,4.3175,4.3811
2008-09-25,1.8906,1.8432,1.8297,1.8746,1.9461,2.5132,3.0046,3.4665,3.7206,4.0012,4.3261,4.3466,4.4164


In [3]:
### Split into smaller dfs based on tenor
small_df = sofr_df.iloc[:,0:5]
big_df = sofr_df.iloc[:,4:]

### Calculate risk free dfs for tenors <= 1Y
tenor_dict = {'1m':1/12, '3m':0.25, '6m':0.5, '9m':0.75, '12m':1, '2y':2, 
              '3y':3, '5y':5, '7y':7, '10y':10, '15y':15, '20y':20, '30y':30}

small_disc = pd.DataFrame(columns = small_df.columns)
for tenor in small_df.columns:
    small_disc[tenor] = 1 / (1 + (tenor_dict[tenor] * small_df[tenor] / 100))
small_disc.head(2)

,1m,3m,6m,9m,12m
dates,,,,,
2008-09-19,0.998385,0.995086,0.990174,0.984897,0.979345
2008-09-25,0.998427,0.995413,0.990934,0.986135,0.980911


In [4]:
### Implementing cubic spline on the entire dataset
time_in_months = [12, 24, 36, 60, 84, 120, 180, 240, 360]
curve_years = [4,6,8,9,11,12,13,14,16,17,18,19,21,22,23,24,25,26,27,28,29]
curve_months = [yr * 12 for yr in curve_years]
spline_cols = ['4y','6y','8y','9y','11y','12y','13y','14y',
               '16y','17y','18y','19y','21y','22y','23y','24y','25y','26y','27y','28y','29y']
spline_df = pd.DataFrame(columns = spline_cols, index = sofr_df.index)

for i in big_df.index:
    rate_sample = big_df.loc[i,:]
    cs = CubicSpline(np.array(time_in_months), rate_sample)
    spline_df.loc[i,:] = [round(py,4) for py in cs(curve_months)]
spline_df.tail(2)

,4y,6y,8y,9y,11y,12y,13y,14y,16y,17y,...,19y,21y,22y,23y,24y,25y,26y,27y,28y,29y
dates,,,,,,,,,,,,,,,,,,,,,
2022-04-21,2.7747,2.7267,2.6941,2.6832,2.6704,2.6672,2.6647,2.6621,2.6526,2.6449,...,2.6241,2.5971,2.5817,2.5651,2.5476,2.5293,2.5103,2.4908,2.4709,2.4508
2022-04-22,2.7352,2.6894,2.674,2.6688,2.6642,2.6643,2.6646,2.6643,2.6581,2.6514,...,2.6316,2.6046,2.589,2.5722,2.5544,2.5358,2.5167,2.4972,2.4775,2.4579


In [5]:
### Create final Swap df
swap_df = pd.concat([sofr_df, spline_df], axis = 1)
swap_cols = ['12m','2y','3y','4y','5y','6y','7y','8y','9y','10y','11y','12y','13y','14y','15y',
               '16y','17y','18y','19y','20y','21y','22y','23y','24y','25y','26y','27y','28y','29y','30y']
swap_df = swap_df[swap_cols]
swap_df.head(2)

,12m,2y,3y,4y,5y,6y,7y,8y,9y,10y,...,21y,22y,23y,24y,25y,26y,27y,28y,29y,30y
dates,,,,,,,,,,,,,,,,,,,,,
2008-09-19,2.1091,2.5107,2.9136,3.2096,3.4175,3.5806,3.7125,3.8195,3.9081,3.9847,...,4.3136,4.308,4.3022,4.2977,4.2959,4.2984,4.3067,4.3222,4.3465,4.3811
2008-09-25,1.9461,2.5132,3.0046,3.2996,3.4665,3.601,3.7206,3.8245,3.9164,4.0012,...,4.3288,4.3096,4.2917,4.2776,4.2699,4.2711,4.2837,4.3105,4.3538,4.4164


In [7]:
### Calculate riskless discount factor
big_disc = small_disc.loc[:,'12m'].to_frame()
for i in range(1,len(swap_df.columns)):
    temp_name = str(i+1)+'y'
    temp_sum = big_disc.iloc[:,0:i].sum(axis=1)
    big_disc[temp_name] = (1 - temp_sum * (swap_df.iloc[:,i])/100) / (1 + (swap_df.iloc[:,i])/100)
big_disc = big_disc.iloc[:,1:]
big_disc.tail(2)

,2y,3y,4y,5y,6y,7y,8y,9y,10y,11y,...,21y,22y,23y,24y,25y,26y,27y,28y,29y,30y
dates,,,,,,,,,,,,,,,,,,,,,
2022-04-21,0.948261,0.920699,0.896099,0.87303,0.850891,0.829427,0.808545,0.788171,0.768225,0.748674,...,0.585888,0.573537,0.561866,0.550823,0.540382,0.530517,0.521182,0.512348,0.503964,0.495996
2022-04-22,0.948476,0.921525,0.897523,0.87490,0.852775,0.830987,0.809745,0.789056,0.768824,0.748982,...,0.584641,0.572311,0.560665,0.549665,0.539269,0.529416,0.520075,0.511196,0.502707,0.494572


In [9]:
### Complete riskless discount factors
riskless_df = pd.concat([small_disc, big_disc], axis = 1)
riskless_df.to_csv('RisklessDF.csv')
riskless_df.head(2)

,1m,3m,6m,9m,12m,2y,3y,4y,5y,6y,...,21y,22y,23y,24y,25y,26y,27y,28y,29y,30y
dates,,,,,,,,,,,,,,,,,,,,,
2008-09-19,0.998385,0.995086,0.990174,0.984897,0.979345,0.951522,0.917024,0.880339,0.843753,0.807386,...,0.402284,0.386413,0.371267,0.356599,0.342169,0.3277,0.312925,0.29759,0.28141,0.264118
2008-09-25,0.998427,0.995413,0.990934,0.986135,0.980911,0.951436,0.914465,0.877125,0.841731,0.806547,...,0.402427,0.388342,0.374797,0.361392,0.347695,0.333278,0.317701,0.30048,0.28121,0.259418
